In [1]:
# https://github.com/scienceai/neocortex/blob/master/examples/cifar10_cnn/cifar10_cnn_example.ipynb

import sys
sys.path.append('./modules/')

# from customLayers import conv_2d_BN, fully_connected_BN
from testing import testModel
from helpers import prepareRun 
import tflearn
from tflearn.layers import batch_normalization, local_response_normalization
from tflearn.data_utils import image_preloader
from tflearn.optimizers import momentum, Adam
from tflearn.activations import softmax, relu
from tflearn.layers.core import input_data, dropout
from tflearn.layers import max_pool_2d, regression, conv_2d, fully_connected
from tflearn.data_augmentation import * 
from tflearn.data_preprocessing import *
from tflearn.data_utils import LabelPreloader, ImagePreloader, Preloader
tf.GraphKeys.LAYER_TENSOR = 'layer_tensor'

In [2]:
struct = prepareRun('3-CNN-Cropping')

In [3]:
x, y = image_preloader('./Data/train/training.tsv', categorical_labels=True,
                              image_shape=[32, 32, 3], mode='file', normalize=True)

In [4]:
imgAug = ImageAugmentation()
imgAug.add_random_flip_leftright()
imgAug.add_random_flip_updown()
imgAug.add_random_crop((28, 28))

imgPrep = ImagePreprocessing()
imgPrep.add_crop_center((28, 28))
# Model constructor
def buildModel():
    network = input_data(shape=[None, 28, 28, 3], data_augmentation=imgAug, data_preprocessing=imgPrep)
    # CNN Block 1
    network = conv_2d(network, 32, 3)
    network = relu(network)
    network = conv_2d(network, 32, 3)
    network = relu(network)
    network = max_pool_2d(network, 2, 2)
    network = dropout(network, 0.7)
    
    # CNN Block 2
    network = conv_2d(network, 64, 3)
    network = relu(network)
    network = conv_2d(network, 64, 3)
    network = relu(network)
    network = max_pool_2d(network, 2, 2)
    network = dropout(network, 0.7)
    
    # CNN Block 3
    network = conv_2d(network, 128, 3)
    network = relu(network)
    network = conv_2d(network, 128, 3)
    network = relu(network)
    network = max_pool_2d(network, 2, 2)
    
    # FC Block 1
    network = dropout(network, 0.5)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='Adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    return network
# Initialize the model and build the trainer:
net = buildModel()

In [5]:
model = tflearn.DNN(net, max_checkpoints=10, tensorboard_verbose=1,
                    checkpoint_path=struct['Checkpoints'], tensorboard_dir=struct['Tensorboard'])

# Train the model - keep 10% (or 5000 images) as a validation set to gauge training effectiveness
model.fit(x, y, n_epoch=128, 
              validation_set=0.10, 
              shuffle=True, 
              show_metric=True, 
              batch_size=64,
              snapshot_step=1000, 
              snapshot_epoch=False, run_id=struct['run'])

model.save(struct['Saves'] + struct['run'])

results = testModel(model, './Data/test/*.png', struct['base'], struct['run'], save_to_csv=True)

Training Step: 90112  | total loss: 0.82900
| Adam | epoch: 128 | loss: 0.82900 - acc: 0.7041 -- iter: 45000/45000
Starting predictions
Predictions done, saving file.


In [ ]:
# https://www.kaggle.com/c/cifar-10/leaderboard?submissionId=3621718
# 0.74420